# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
test = pd.read_csv('flights_test.csv')

In [3]:
df = pd.read_csv('flights_merged.csv', index_col=0)

In [4]:
df = df.dropna(axis=0, subset=['crs_elapsed_time'])

In [5]:
df = df.sample(frac=0.1)

In [6]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,passengers,capacity,total_gallons,dep_delay,nas_delay,security_delay,late_aircraft_delay,weather_delay,carrier_delay
29989,2018-01-07,WN,WN,WN,3566,WN,N470WN,3566,11259,DAL,...,460.0,16217.500000,3659.500000,1.723195e+08,37.111111,1.444444,0.000000,19.333333,1.222222,11.388889
30882,2019-01-03,WN,WN,WN,1261,WN,N7734H,1261,13495,MSY,...,471.0,9228.000000,3499.000000,1.723195e+08,27.454545,1.272727,2.272727,10.272727,0.000000,9.272727
7374,2019-01-05,AA,AA_CODESHARE,AA,6020,CP,N200NN,6020,12892,LAX,...,337.0,2006.333333,205.666667,0.000000e+00,37.307692,22.179487,0.000000,11.589744,0.000000,10.358974
13595,2018-01-03,AA,AA,AA,1168,AA,N957NN,1168,13495,MSY,...,837.0,3732.666667,1120.333333,3.079852e+08,57.750000,6.250000,0.000000,23.000000,0.000000,30.750000
11718,2018-01-01,AA,AA,AA,1312,AA,N939AN,1312,11057,CLT,...,361.0,10033.500000,1289.500000,3.079852e+08,35.076923,1.923077,0.000000,13.461538,0.000000,14.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15716,2019-01-06,AS,AS,AS,134,AS,N471AS,134,10299,ANC,...,1542.0,1861.750000,573.250000,6.101888e+07,31.000000,4.200000,0.000000,8.800000,20.400000,3.200000
8184,2018-01-07,AA,AA,AA,2518,AA,N984NN,2518,13930,ORD,...,802.0,12024.090909,1640.090909,3.079852e+08,49.807692,9.269231,0.000000,15.519231,5.307692,22.692308
9035,2019-01-02,AA,AA,AA,1064,AA,N149AN,1064,11298,DFW,...,1171.0,33326.500000,3943.500000,3.079852e+08,42.171429,6.571429,0.000000,15.057143,0.942857,22.514286
23798,2019-01-01,WN,WN,WN,398,WN,N413WN,398,11259,DAL,...,562.0,6840.000000,1661.000000,1.723195e+08,34.153846,2.846154,0.000000,14.461538,0.000000,15.000000


In [7]:
y= df['arr_delay']

In [8]:
X = df

In [9]:
X = X.drop(columns=['arr_delay'])

In [10]:
X.fl_date = X.fl_date.apply(pd.to_datetime)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [11]:
X = X[['origin_airport_id', 'dest_airport_id', 'distance', 'passengers', 'crs_elapsed_time','capacity', 'total_gallons', 'dep_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay',
       'weather_delay', 'carrier_delay']]
#dropped flight date :( 

In [12]:
x_label = X.columns

In [13]:
scaler = StandardScaler()

In [14]:
X = pd.DataFrame(scaler.fit_transform(X))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8)


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [215]:
#data_dmatrix = xgb.DMatrix(data=X,label=y, enable_categorical=True)

In [216]:
#objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 7, alpha = 0.1,

In [16]:
lr = LinearRegression()

In [17]:
lr.fit(X_train, y_train) 

LinearRegression()

In [18]:
lr.score(X_train, y_train)

0.159267945741012

In [19]:
lr.score(X_test, y_test)

0.17962858113333258

### Random Forest

RandomForestRegressor()

In [20]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100)
 

In [21]:
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor()

In [22]:
y_pred = regressor.predict(X_test)

In [23]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 29.54027670901269
Mean Squared Error: 1643.2005992991765
Root Mean Squared Error: 40.53641078461654


In [24]:
feature_importance= pd.Series(regressor.feature_importances_, index= x_label).sort_values(ascending=False)
feature_importance

dep_delay              0.220665
crs_elapsed_time       0.182979
nas_delay              0.090361
late_aircraft_delay    0.068881
carrier_delay          0.065717
passengers             0.063864
capacity               0.061855
origin_airport_id      0.059212
dest_airport_id        0.058391
distance               0.049645
total_gallons          0.042902
weather_delay          0.029439
security_delay         0.006089
dtype: float64

In [25]:
params = {'n_estimators' : [100, 150, 200, 300], 'max_depth': [3, 10, 20, 30]}

In [26]:
grid = GridSearchCV(regressor, params)

In [27]:
grid_result = grid.fit(X_train, y_train)

In [28]:
print(grid_result.best_score_)
print( grid_result.best_params_['n_estimators'])
print( grid_result.best_params_['max_depth'])

0.11898671987936302
100
3


### XGBOOST

In [29]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 7, alpha = 0.1,)

In [ ]:
xg_reg.fit(X_train,y_train)


In [ ]:
preds = xg_reg.predict(X_test)

In [ ]:
xg_reg.score(X_train,y_train)

In [ ]:
xg_reg.score(X_test,y_test)

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.